In [2]:
import numpy as np
import pandas as pd
import os
import json
from tqdm.notebook import tqdm
import tiktoken

In [ ]:
root = "../data/cases_json/"

n_cases = len(os.listdir(root))
n_cases

471

In [4]:
case_ids = np.zeros((n_cases), dtype='object')
file_names = np.zeros((n_cases), dtype='object')
outcomes = np.zeros((n_cases), dtype='object')

In [5]:
for i, file_name in enumerate(tqdm(sorted(os.listdir(root)))):
    case_ids[i] = file_name[:4]
    file_names[i] = file_name
    
    case_file_path = root + file_name
    with open(case_file_path, encoding='utf-8') as f:
        case_file = json.load(f)
    
    outcomes[i] = case_file['class']

  0%|          | 0/471 [00:00<?, ?it/s]

In [6]:
metadata_df = pd.DataFrame({'case_id': case_ids,
                            'file_name': file_names,
                            'outcome': outcomes})
metadata_df.head()

,case_id,file_name,outcome
0,0001,0001_Caselaw_AZ_1490855-Sasha.json,reverse
1,0002,0002_Caselaw_Alaska_11695007-Sasha.json,affirm
2,0029,0029-nm_119_609-Torres v State-Sasha.json,reverse
3,0032,0032-nm_118_685-Baer v Regents of the Universi...,mixed
4,0033,0033-nm_118_385-Diaz ex rel Diaz v Feil-Sasha....,mixed


In [7]:
metadata_df.to_csv("../data/csvs/metadata.csv", index=False)

In [8]:
### Case Lengths
encoding = tiktoken.encoding_for_model('gpt-4o')
token_lengths = np.zeros((n_cases, 6))
for i in tqdm(range(n_cases)):
    file_path = root + metadata_df['file_name'][i]
    with open(file_path, encoding='utf-8') as f:
        case_file = json.load(f)

    token_lengths[i, 0] = len(encoding.encode(case_file['text']))
    token_lengths[i, 1] = len(encoding.encode(" ".join(case_file['annotations']['Facts'])))
    token_lengths[i, 3] = len(encoding.encode(" ".join(case_file['annotations']['Relevant Precedents'])))
    token_lengths[i, 2] = len(encoding.encode(" ".join(case_file['annotations']['Procedural History'])))
    token_lengths[i, 4] = len(encoding.encode(" ".join(case_file['annotations']['Application of Law to Facts'])))
    token_lengths[i, 5] = len(encoding.encode(" ".join(case_file['annotations']['Outcome'])))
token_lengths_df = pd.DataFrame(token_lengths, index=metadata_df['file_name'], columns=['text', 'facts', 'procedural_history', 'relevant_precedents', 'application_of_law_to_facts', 'outcome'])
token_lengths_df.head()
token_lengths_df.describe()
# token_lengths_df.reset_index().to_csv("../data/token_lengths.csv", index=False)

  0%|          | 0/471 [00:00<?, ?it/s]

,text,facts,procedural_history,relevant_precedents,application_of_law_to_facts,outcome
count,471.000000,471.000000,471.000000,471.000000,471.000000,471.000000
mean,5035.373673,620.148620,134.138004,164.280255,340.254777,29.968153
std,3104.940872,603.597346,125.452305,229.445785,235.630288,31.904129
min,802.000000,0.000000,0.000000,0.000000,20.000000,0.000000
25%,2935.000000,274.000000,41.500000,29.500000,175.500000,10.000000
50%,4433.000000,461.000000,102.000000,83.000000,281.000000,21.000000
75%,6318.000000,743.000000,193.000000,208.000000,438.000000,37.000000
max,23125.000000,5150.000000,761.000000,2057.000000,1726.000000,242.000000
